In [1]:
import os
import h5py
import numpy as np
import requests
import cStringIO
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

# path to the model weights file.
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
# dimensions of our images.
img_width, img_height = 224, 224

with open('synsets.txt') as f:
    classes = [entry.strip() for entry in f.readlines()]

# build the VGG16 network
model = Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(img_width, img_height, 3), activation='relu', border_mode='same', name='block1_conv1'))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1'))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

# Classification Block
model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dense(len(classes), activation='softmax', name='predictions'))

Using TensorFlow backend.


In [2]:
model.load_weights(weights_path)

In [3]:
## helper functions
def load_image_from_url(image_url):
    request_respone = requests.get(image_url).content
    image = Image.open(cStringIO.StringIO(request_respone))
    return image

def preprocess_for_vgg(image):
    image = image.resize([224,224])
    x = np.array(image, dtype=float)
    x_fake_batch = x.reshape(1,*x.shape)
    x = x_fake_batch
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    return x

In [4]:
url = 'http://i.huffpost.com/gen/1106361/images/h-AIRPLANES-MOBILE-628x314.jpg'
numpy_array = preprocess_for_vgg(load_image_from_url(url))
predictions = model.predict(numpy_array, batch_size=numpy_array.shape[0])[0]
top_prediction = np.argmax(predictions)
classes[top_prediction]

'n02690373 airliner'

In [5]:
url = 'http://dreamatico.com/data_images/kitten/kitten-1.jpg'
numpy_array = preprocess_for_vgg(load_image_from_url(url))
predictions = model.predict(numpy_array, batch_size=numpy_array.shape[0])[0]
top_prediction = np.argmax(predictions)
classes[top_prediction]

'n02124075 Egyptian cat'